In [24]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
import pickle

In [25]:
#this function load the RGB image and change to gray inage, change the 2D image data to 1D and form a matrix
def load_image_to_vector(image_path,image_number):
    img_data=[]
    for i in range (image_number):
        img_color= mpimg.imread(image_path + str(i) + '.jpg')
        img=cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY)
        width=np.shape(img_color)[0]
        height=np.shape(img_color)[1]
        img_vector=img.reshape(width*height)
        img_data.append(img_vector)
    return img_data

In [31]:
#PCA is used for dimensionality reduction
def img_data_pca(img_data,dimention):
    pca = PCA(n_components = 100)
    pca.fit(img_data)
    pca_data=pca.transform(img_data)
    return pca_data

In [27]:
def get_label(label_path,label_name):
    label=pd.read_table(label_path)
    y=label[label_name]
    return y

In [37]:
def LogisticRegression_model(x_train,y_train,x_test,y_test):
    clf = LogisticRegression(solver='lbfgs',fit_intercept=True,max_iter=1000)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print('Accuracy on train set:'+str(clf.score(x_train,y_train)))
    print('Accuracy on test set: '+str(accuracy_score(y_test,y_pred)))
    print(classification_report(y_test,y_pred))
    return clf

In [39]:
#get image data
img_data_train=load_image_to_vector('../datasets/celeba/img/',5000)
img_data_test=load_image_to_vector('../datasets/celeba_test/img/',1000)
#pca for image data
x_train=img_data_pca(img_data_train,100)
x_test=img_data_pca(img_data_test,100)
#get label
y_train=get_label('../datasets/celeba/labels.csv','gender')
y_test=get_label('../datasets/celeba_test/labels.csv','gender')
#Standardize the data
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)
#train the model and report accuracy
model=LogisticRegression_model(x_train,y_train,x_test,y_test)
#save model
pickle.dump(model,open("LogisticRegression_gender.dat","wb")) 

Accuracy on train set:0.8062
Accuracy on test set: 0.626
              precision    recall  f1-score   support

          -1       0.63      0.62      0.62       500
           1       0.62      0.63      0.63       500

    accuracy                           0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000



In [9]:
#save model
pickle.dump(logRegr,open("LogisticRegression_gender_separate.dat","wb"))  
#load model
loaded_model = pickle.load(open("LogisticRegression_gender_separate.dat","rb"))
y_pred=loaded_model.predict(x_test)